In [1]:
import sklearn
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVR
import h5py
from sklearn.metrics import mean_squared_error
import math

In [2]:
fname = 'BJ{}_M32x32_T30_InOut.h5'.format(16)
reader=h5py.File(fname,'r')
data=reader['data'].value

In [3]:
def window_stack(x, stepsize=1, window=3, pad=1):                                                    
    m,n = x.shape                                                                                    
    res = np.array([[x[i:i+window:stepsize,j:j+window:stepsize] for j in range(n-window+1)] for i in range(m-window+1)])                                                                                 
    m -= 2*pad                                                                                       
    n -= 2*pad                                                                                       
    return res.reshape(m*n,window*window)

In [4]:
stepsize=1
window=3
pad=1
m=n=32
timeslot=100
train_x=[]
train_y=[]
test_x=[]
test_y=[]
for k in range(timeslot):
    if( k==timeslot-1):
        break
    for i in range(m-window+1):
        for j in range(n-window+1):
            train_x.append(data[k][1][i:i+window:stepsize,j:j+window:stepsize])
            train_y.append(data[k+1][1][i,j])

In [5]:
train_x=np.asarray(train_x)
train_x=train_x.reshape((timeslot-1)*900,9)
train_x.shape

(179100, 9)

In [6]:
train_y=np.array(train_y)
train_y=train_y.reshape(-1,1)
train_y.shape

(179100, 1)

In [7]:
minmaxx=preprocessing.MinMaxScaler()
minmaxy=preprocessing.MinMaxScaler()
train_x=minmaxx.fit_transform(train_x)
train_y=minmaxy.fit_transform(train_y)

In [8]:
train_len=int(0.7*len(train_x))
print()

In [9]:
test_x=train_x[train_len:]
train_x=train_x[0:train_len]
test_y=train_y[train_len:]
train_y=train_y[0:train_len]

In [10]:
%%time
rbf_svr = SVR(kernel='rbf')
rbf_svr.fit(train_x, train_y)
print(1)

C:\Users\hu\Miniconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1
Wall time: 13.6 s


In [11]:
predict_y=rbf_svr.predict(test_x)

In [12]:
predict_y=predict_y.reshape(-1,1)

In [13]:
predict_y=minmaxy.inverse_transform(predict_y)
test_y=minmaxy.inverse_transform(test_y)

In [14]:
print(predict_y[0:10])
print(test_y[0:10])

[[122.86513612]
 [ 80.93825673]
 [ 96.47624204]
 [132.05951795]
 [ 91.20682899]
 [ 94.87648439]
 [ 89.83178333]
 [167.86090069]
 [386.32771858]
 [339.20262561]]
[[ 46.]
 [  0.]
 [ 43.]
 [120.]
 [ 49.]
 [ 90.]
 [101.]
 [160.]
 [438.]
 [346.]]


In [15]:
print(math.sqrt(mean_squared_error(predict_y,test_y)))

56.75918166347087
